**Compute ECF using altered ehfheatwaves library**

Install ehfheatwaves library from the zip file onto a folder, and open the terminal on that directory. Do pip install like:

In [2]:
#pip install .

Run this on command line (the mask is not necessary):

In [40]:
#ehfheatwaves -x /path/to/tx.nc --vnamex tx -n /path/to/tn.nc --vnamen tn -m  /path/to/mask.nc --vnamem mask --base=1991-2020 --daily --dailyonly --ecf -p 10

Analyze ehf

In [ ]:
import xarray as xr
ds_ecf = xr.open_dataset('/path/to/ecf.nc')
ds_ecf = ds_ecf[['ECF']]
ds_ecf.ECF.isel(time=-1).plot()

To create a video, choose a timeline:

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pathlib import Path
#from matplotlib.colors import TwoSlopeNorm

# Functions for creating and rendering video
def render_video(output_video_filepath, frames, fps=5):
    # Parameters
    width, height = frames[0].shape[1], frames[0].shape[0]  # Use the size of the first frame

    # Create VideoWriter object
    out = cv2.VideoWriter(output_video_filepath, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Write frames to video
    for frame in frames:
        out.write(frame)

    # Release VideoWriter
    out.release()

def create_video_frames(figs):
    frames = []
    for fig in figs:
        canvas = FigureCanvas(fig)
        canvas.draw()  # Draw the figure
        img = np.array(canvas.renderer.buffer_rgba())  # Convert to RGBA buffer
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)  # Convert to BGR (OpenCV) format
        frames.append(img_bgr)
        plt.close(fig)  # Close the figure to free memory
    return frames

def dist_value_range(dataarray, n_dev=1):
    """
    Calculate a smart value range for a DataArray based on standard deviations
    from the mean.

    Parameters:
        dataarray (xarray.DataArray): The input DataArray.

    Returns:
        tuple: (vmin, vmax) for the colorbar range.
    """
    mean = dataarray.mean(skipna=True).item()
    std = dataarray.std(skipna=True).item()
    vmin = mean - n_dev * std
    vmax = mean + n_dev * std
    return vmin, vmax

def mk1frame(i, dataarray, value_range,
              title = None, dist_range= False, ):

    if value_range is None:
        if dist_range: vmin, vmax = dist_value_range(dataarray)
        else: vmin, vmax = float(dataarray.min()), float(dataarray.max())
    else: vmin, vmax = value_range

    fig, ax = plt.subplots(figsize=(10, 6))
    dataarray.isel(time=i).plot(ax=ax, vmin=vmin, vmax=vmax,
                                 cbar_kwargs={"shrink": 0.7},
                                 #cmap="coolwarm",
                                 #norm=TwoSlopeNorm(vcenter=0,vmin=vmin, vmax=vmax)
                                    )
    
    if title is None: title = dataarray.name
    date = dataarray.time[i].dt.strftime("%Y-%m-%d").item()
    ax.set_title(f"{title} - {date}")

    return fig

# Function to create an animation from an xarray DataArray
def create_video_from_xarray(dataarray, output_video_filepath, value_range=None, title=None, fps=5, dist_range = False):
    output_path = Path(output_video_filepath)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    if not output_video_filepath.endswith('.mp4'):
        raise ValueError("Output file path must end with '.mp4'")

    figs = []
    for i in range(len(dataarray.time)):
        fig = mk1frame(i, dataarray, value_range, title, dist_range)
        #plt.show(fig)
        figs.append(fig)

    # Convert figures to frames
    frames = create_video_frames(figs)
    del figs; del fig

    # Save video
    render_video(output_video_filepath, frames, fps); del frames

In [ ]:
output_file = './ds_ecf.mp4'
var_name='ECF'

date1 = '2018-01-01'
date2 = '2018-12-31'

datarray = ds_ecf[var_name].sel(time=slice(date1, date2))

create_video_from_xarray(datarray, output_file, value_range=None, title='ECF', dist_range=False)